In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor


# read file
val = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('training_extra.csv')

# lable splite
xtrain = train.drop('Price', axis = 1)
ytrain = train['Price']

xval = val.drop('Price', axis=1)
yval = val['Price']

#saparating numeric and non numeric
xtrain_num = xtrain.select_dtypes(include=[np.number])
xtrain_cat = xtrain.select_dtypes(exclude=[np.number])

xval_num = xval.select_dtypes(include=[np.number])
xval_cat = xval.select_dtypes(exclude=[np.number])

test_num = test.select_dtypes(include=[np.number])
test_cat = test.select_dtypes(exclude=[np.number])

# standerdization
scaler = StandardScaler()
Xtrain_num = scaler.fit_transform(xtrain_num)
Xval_num = scaler.fit_transform(xval_num)
test_num = scaler.fit_transform(test_num)

#encoding
encoder = OneHotEncoder()
xtrain_cat = encoder.fit_transform(xtrain_cat)
xval_cat = encoder.fit_transform(xval_cat)
test_cat = encoder.fit_transform(test_cat)

#combining
Xtrain = np.hstack((Xtrain_num, xtrain_cat.toarray()))
Xval = np.hstack((Xval_num, xval_cat.toarray()))
Xtest = np.hstack((test_num, test_cat.toarray()))

In [ ]:
#print(np.isnan(ytrain).sum())
ytrain = np.nan_to_num(ytrain, nan=np.mean(ytrain))
yval = np.nan_to_num(yval, nan=np.mean(yval))

In [ ]:
print(xtrain_num)

             id  Compartments  Weight Capacity (kg)
0        500000          10.0             23.882052
1        500001           4.0             11.869095
2        500002           8.0              8.092302
3        500003           7.0              7.719581
4        500004           9.0             22.741826
...         ...           ...                   ...
683044  1183044           7.0             28.369664
683045  1183045           8.0              6.948439
683046  1183046           2.0             23.899760
683047  1183047           1.0             18.611359
683048  1183048           NaN                   NaN

[683049 rows x 3 columns]


In [ ]:
#train model
rfr = RandomForestRegressor()
rfr.fit(Xtrain, ytrain)


RandomForestRegressor()

In [ ]:
pred = rfr.predict(Xval)
print('predicted data: ',pred)
print('actual data: ',yval)

predicted data:  [82.1864085 82.6468399 73.8567343 ... 95.4963731 83.3005473 60.7155762]
actual data:  [112.15875  68.88056  39.1732  ... 111.41364 115.8908   26.72762]


In [ ]:
#root mean sqr error
rmse_np = np.sqrt(np.mean((yval - pred) ** 2))
print(f"RMSE: {rmse_np:.3f}")

RMSE: 39.699


In [ ]:
test_df = pd.DataFrame(test)
pred_series = pd.Series(pred, name="Predicted").round(3)
id_column = test_df.iloc[:, 0]
final_df = pd.concat([id_column, pred_series], axis=1)
final_df.to_csv("submission.csv", index=False)
print("Final CSV file with 'ID' and 'Predicted' saved successfully!")

Final CSV file with 'ID' and 'Predicted' saved successfully!
